In [1]:
import re
import pandas as pd

text = "1879-03-14  1879年 3月14日 ドイツ帝国 ヴュルテンベルク王国 ウルム"

# 日付の抽出 (YYYY-MM-DD または YYYY年 MM月 DD日)
date_match = re.search(r"(\d{4}年\s*\d{1,2}月\s*\d{1,2}日)", text)
if date_match:
    date_str = date_match.group(1) or date_match.group(2)
else:
    date_str = None

# 出生地の抽出
place_match = re.search(r"(.+)", text)
if place_match:
    place_str = place_match.group(1)
    #日付を削除
    place_str = re.sub(r"(\d{4}-\d{2}-\d{2})|(\d{4}年\s*\d{1,2}月\d{1,2}日)", '', place_str).strip()
else:
    place_str = None


print(f"日付: {date_str}")
print(f"出生地: {place_str}")

日付: 1955年 4月18日
出生地: 76歳没 アメリカ合衆国 ニュージャージー州 プリンストン


In [12]:
import dateparser
# 日付のフォーマットを統一し、年、月、日に分解
if date_str:
    parsed_date = dateparser.parse(date_str, date_formats=['%Y-%m-%d', '%Y年%m月%d日'])
    if parsed_date:
      birth_year = parsed_date.year
      birth_month = parsed_date.month
      birth_day = parsed_date.day
    else:
      birth_year = None
      birth_month = None
      birth_day = None
else:
    birth_year = None
    birth_month = None
    birth_day = None

print(f"生年: {birth_year}")
print(f"生月: {birth_month}")
print(f"生日: {birth_day}")

生年: 1879
生月: 3
生日: 14


In [13]:
# 出生地の分解
if place_str:
    parts = place_str.split()
    if len(parts) >= 3:
      birth_country = parts[0]
      birth_kingdom = parts[1]
      birth_city = parts[2]
    elif len(parts) >= 2:
      birth_country = parts[0]
      birth_kingdom = None
      birth_city = parts[1]
    else:
      birth_country = None
      birth_kingdom = None
      birth_city = parts[0]

else:
    birth_country = None
    birth_kingdom = None
    birth_city = None

print(f"出生国: {birth_country}")
print(f"出生王国: {birth_kingdom}")
print(f"出生都市: {birth_city}")

出生国: ドイツ帝国
出生王国: ヴュルテンベルク王国
出生都市: ウルム


In [14]:
# DataFrame の作成
data = {
    "birth_year": [birth_year],
    "birth_month": [birth_month],
    "birth_day": [birth_day],
    "birth_country": [birth_country],
    "birth_kingdom": [birth_kingdom],
    "birth_city": [birth_city],
}
df = pd.DataFrame(data)

print(df)

   birth_year  birth_month  birth_day birth_country birth_kingdom birth_city
0        1879            3         14         ドイツ帝国    ヴュルテンベルク王国        ウルム


In [8]:
import re
import pandas as pd
import dateparser
from utils.logger import get_logger

logger = get_logger(__name__)


def extract_and_format_death_info(text):
    """
    _data_processor.py の _extract_text_from_cell メソッドで抽出されたテキストから、
    key_type の "death_date" に基づいて、死亡年月日、年齢、死亡地の情報を詳細に抽出し、DataFrame を作成する。

    Args:
        text (str): 処理対象のテキスト（例: "1955-04-18  1955年 4月18日 76歳没 アメリカ合衆国 ニュージャージー州 プリンストン"）

    Returns:
        pandas.DataFrame: 死亡年月日、年齢、死亡地の情報を格納した DataFrame。
                          テキストから情報が抽出できなかった場合は空の DataFrame を返す。
    """
    logger.debug(f"extract_and_format_death_info: 処理対象テキスト: {text}")

    # 日付の抽出 (YYYY年 MM月 DD日 または YYYY-MM-DD)
    date_match = re.search(r"(\d{4})年\s*(\d{1,2})月(\d{1,2})日|(\d{4})-(\d{1,2})-(\d{1,2})", text)
    if date_match:
        if date_match.group(1):
            year_str, month_str, day_str = date_match.groups()[:3]
            date_str = f"{year_str}年{month_str}月{day_str}日"
        else:
            year_str, month_str, day_str = date_match.groups()[3:]
            date_str = f"{year_str}年{month_str}月{day_str}日"

    else:
        date_str = None

    # 死亡年齢の抽出
    age_match = re.search(r"(\d+)歳没", text)
    age_str = age_match.group(1) if age_match else None

    # 死亡地の抽出
    place_match = re.search(r"(.+)$", text)
    if place_match:
        place_str = place_match.group(1)
        # 日付と年齢を削除
        place_str = re.sub(r"(\d{4})年\s*(\d{1,2})月(\d{1,2})日|(\d{4})-(\d{1,2})-(\d{1,2})|\d+歳没", "", place_str).strip()
    else:
        place_str = None

    if not date_str:
        logger.warning("extract_and_format_death_info: 死亡年月日の抽出に失敗しました。")
        if age_str:
            logger.warning(f"extract_and_format_death_info: 死亡年齢: {age_str}")
        if place_str:
            logger.warning(f"extract_and_format_death_info: 死亡地: {place_str}")
        return pd.DataFrame()

    logger.debug(f"extract_and_format_death_info: 死亡年月日 (抽出): {date_str}")
    if age_str:
        logger.debug(f"extract_and_format_death_info: 死亡年齢: {age_str}")
    if place_str:
        logger.debug(f"extract_and_format_death_info: 死亡地: {place_str}")

    # 日付のフォーマットを統一し、年、月、日に分解
    parsed_date = dateparser.parse(date_str, date_formats=['%Y年%m月%d日'])
    if parsed_date:
        death_year = parsed_date.year
        death_month = parsed_date.month
        death_day = parsed_date.day
    else:
        logger.warning("extract_and_format_death_info: 死亡年月日の解析に失敗しました。")
        return pd.DataFrame()

    logger.debug(f"extract_and_format_death_info: 死亡年: {death_year}, 死亡月: {death_month}, 死亡日: {death_day}")

    # 死亡地の分解
    death_country, death_state, death_city = None, None, None

    if place_str:
        parts = place_str.split()
        if len(parts) >= 3:
            death_country = parts[0]
            death_state = parts[1]
            death_city = parts[2]
        elif len(parts) >= 2:
            death_country = parts[0]
            death_state = parts[1]
            death_city = None
        elif len(parts) == 1:
            death_city = parts[0]
        logger.debug(f"extract_and_format_death_info: 死亡国: {death_country}, 死亡州: {death_state}, 死亡都市: {death_city}")

    # DataFrame の作成
    data = {
        "death_year": [death_year],
        "death_month": [death_month],
        "death_day": [death_day],
        "death_age": [int(age_str) if age_str else None],
        "death_country": [death_country],
        "death_state": [death_state],
        "death_city": [death_city],
    }
    df = pd.DataFrame(data)
    logger.debug(f"extract_and_format_death_info: 作成したDataFrame:\n{df}")

    return df

# テスト用のテキスト
text = "1955-04-18  1955年 4月18日 76歳没 アメリカ合衆国 ニュージャージー州 プリンストン"
# テスト用のテキスト2(都市のみ)
text2 = "1955-04-18  1955年 4月18日 76歳没 プリンストン"
# テスト用のテキスト3(州のみ)
text3 = "1955-04-18  1955年 4月18日 76歳没 ニュージャージー州"
# テスト用のテキスト4(日付なし)
text4 = "76歳没 アメリカ合衆国 ニュージャージー州 プリンストン"

# 関数を実行
df = extract_and_format_death_info(text)
print("test1")
print(df)

df2 = extract_and_format_death_info(text2)
print("test2")
print(df2)

df3 = extract_and_format_death_info(text3)
print("test3")
print(df3)

df4 = extract_and_format_death_info(text4)
print("test4")
print(df4)

2025-03-14 11:19:20.284 | DEBUG    | __main__:extract_and_format_death_info:21 - extract_and_format_death_info: 処理対象テキスト: 1955-04-18  1955年 4月18日 76歳没 アメリカ合衆国 ニュージャージー州 プリンストン
2025-03-14 11:19:20.285 | DEBUG    | __main__:extract_and_format_death_info:57 - extract_and_format_death_info: 死亡年月日 (抽出): 1955年04月18日
2025-03-14 11:19:20.287 | DEBUG    | __main__:extract_and_format_death_info:59 - extract_and_format_death_info: 死亡年齢: 76
2025-03-14 11:19:20.288 | DEBUG    | __main__:extract_and_format_death_info:61 - extract_and_format_death_info: 死亡地: アメリカ合衆国 ニュージャージー州 プリンストン
2025-03-14 11:19:20.288 | DEBUG    | __main__:extract_and_format_death_info:73 - extract_and_format_death_info: 死亡年: 1955, 死亡月: 4, 死亡日: 18
2025-03-14 11:19:20.289 | DEBUG    | __main__:extract_and_format_death_info:90 - extract_and_format_death_info: 死亡国: アメリカ合衆国, 死亡州: ニュージャージー州, 死亡都市: プリンストン
2025-03-14 11:19:20.291 | DEBUG    | __main__:extract_and_format_death_info:103 - extract_and_format_death_info: 作成したDataFrame:
   

test1
   death_year  death_month  death_day  death_age  death_country      death_state       death_city 
0     1955          4          18         76      アメリカ合衆国  ニュージャージー州  プリンストン
test2
   death_year  death_month  death_day  death_age death_country death_state   death_city 
0     1955          4          18         76          None         None     プリンストン
test3
   death_year  death_month  death_day  death_age death_country death_state      death_city    
0     1955          4          18         76          None         None     ニュージャージー州
test4
Empty DataFrame
Columns: []
Index: []


In [2]:
import re

text = """
アインシュタインは1879年3月14日、 ヘルマン・アインシュタイン を父、 パウリーネ・コッホ を母とし、その長男としてドイツ南西部の バーデンヴュルテンベルク州 ウルム市 にて生まれた 。父ヘルマンはその弟ヤコブから誘われ、アルベルト誕生翌年の1880年夏、一家は ミュンヘン に引っ越し、兄弟は、直流電流に基づいた電気機器を製造する会社Elektrotechnische Fabrik J Einstein Cieを設立した。ヘルマンは営業を担当しヤコブは技術を担当した。1881年には一家にマリアアルベルトの妹。通称マーヤ。が誕生し、一家は1894年まで同地ミュンヘンで暮らすことになる。 アインシュタインは、5歳ごろまであまり言葉を発して他人と会話することがなかった 。しかし、5歳のときに父親からもらった 方位磁針 が、自然界の仕組みに対する興味をもたらすきっかけとなった 。また、同じ頃、 ヴァイオリン を習い始めている 。そしてすぐに モーツァルト の曲が好きになり、ヴァイオリンは生涯の友となった。 アインシュタイン一家はその家系からして アシュケナージ系ユダヤ人 ではあったものの、敬虔なユダヤ教徒というわけではなかったため、アインシュタインは5歳から3年間、ミュンヘンにある カトリック 系の公立学校へ通った。卒業後はミュンヘンの ルイトポルト・ギムナジウム 英語版 現在ではアルバート・アインシュタイン・ギムナジウムと呼ばれている学校に入学。以後7年間、ドイツを離れイタリアに行くまで教育を受ける。しかし、同校の 軍国主義 的で重苦しい校風にはなじめなかった 。 幼少のころは、言葉を理解したり話したりするという面では問題がなかったが、言葉を出すのには時間を要した。一方で数学に関しては傑出した才能を示し、9歳のときに ピタゴラスの定理 の存在を知り、その定理の美しい証明を寝る間も惜しんで考え、そして自力で定理を証明した。12歳のときに家庭教師の マックス・タルムード から、 テオドール・シュピーカー の ユークリッド幾何学 の本をもらい 独習 。 微分 学と 積分 学も、この当時に独学で習得したといわれている。同じころ、医学生だった マックス・タルメイ から 天文学 の存在を知らされ、同時に物理学に関心を示すようになったという。 1894年 、父と叔父の会社が行きづまり 、その結果、新たな商業的な機会を求めて一家は イタリア の ミラノ に引っ越すことになった。父ヘルマンはアインシュタインがギムナジウムをしっかり卒業する必要があると判断し、アインシュタインだけ同地に残されることになった。父ヘルマンはアルベルトが 電気工学 の道へと進むといいと考えていたのだった。だがアインシュタインは規則ずくめで軍国主義的な校風と対立・反発し、1894年12月末、医師に書かせた診断書を口実にして退校を申し出て、家族を追って旅をし、当時イタリアの パヴィア にいた家族のもとへとやってきた 。このイタリアでの滞在中、アインシュタインは磁界中での エーテル の状態の調査についてという題名の短い試論を書いたという。
"""

def extract_parents_info(text):
    father_pattern = re.compile(r'(?P<father>[\w・ー]+)\s*を父')
    mother_pattern = re.compile(r'(?P<mother>[\w・ー]+)\s*を母')

    father_match = father_pattern.search(text)
    mother_match = mother_pattern.search(text)

    father_name = father_match.group('father') if father_match else None
    mother_name = mother_match.group('mother') if mother_match else None

    return {
        'father': father_name,
        'mother': mother_name
    }

parents_info = extract_parents_info(text)
print(parents_info)

{'father': 'ヘルマン・アインシュタイン', 'mother': 'パウリーネ・コッホ'}
